In [1]:
import pandas as pd
import config

def get_data(mode='train'):
    if mode == 'train':
        path = config.PATH_TO_INPUT_TRAIN
    elif mode == 'test':
        path = config.PATH_TO_INPUT_TEST
    else:
        raise NotImplemented
        
    df = (pd.read_csv(path)
             .drop('Name', axis=1)
             .drop_duplicates(subset=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 
                                      'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 
                                      'VRDeck'] + ['Transported' if mode == 'train' else 'VRDeck'],
                              keep='first')
             .reset_index(drop=True)
         )
    # Split Cabin
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if type(x) == str else None)
    df['CabinNum'] = df['Cabin'].apply(lambda x: x.split('/')[1] if type(x) == str else None)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split('/')[2] if type(x) == str else None)
    
    # Drop unused columns
    df = df.drop(['Cabin'], axis=1)
    return df

df = get_data(mode='train')

In [2]:
# Split
TRAIN_PERCENT = 80

df['InTrain'] = df['PassengerId'].apply(lambda x: hash(x) % 100 < TRAIN_PERCENT)
df_train = df.query('InTrain').drop('InTrain', axis=1).reset_index(drop=True)
df_val = df.query('~InTrain').drop('InTrain', axis=1).reset_index(drop=True)

In [4]:
df_train.Transported.value_counts(1)

True     0.505123
False    0.494877
Name: Transported, dtype: float64

In [6]:
df_val.Transported.value_counts(1)

False    0.506575
True     0.493425
Name: Transported, dtype: float64

In [3]:
# Save
df_train.to_csv(config.PATH_TO_TRAIN_SPLIT, index=False)
df_val.to_csv(config.PATH_TO_VAL_SPLIT, index=False)